In [1]:
import sys,os
sys.path.append('..')

In [2]:
import os
import random
import numpy as np
import torch

def set_seed(seed=42):
    """
    全局随机种子，确保结果可复现
    """
    random.seed(seed)                  # Python 内置随机
    np.random.seed(seed)               # Numpy
    torch.manual_seed(seed)            # CPU
    torch.cuda.manual_seed(seed)       # 当前 GPU
    torch.cuda.manual_seed_all(seed)   # 所有 GPU（多卡）
    
    # 确保 CuDNN 可复现
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # 固定 Python 环境的 hash 种子（部分字符串 hash 会影响 DataLoader）
    os.environ["PYTHONHASHSEED"] = str(seed)

    print(f"✅ Global random seed set to {seed}")

# 调用一次
set_seed(42)


✅ Global random seed set to 42


In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from utils.tools import get_setting
from exp.trainer_forecasting import Trainer_Forecasting
from utils.arguments import get_args
from utils.print_args import print_args
from models.ablation_variants import build_variant

In [12]:
df_raw = pd.read_csv('new_ETTh1.csv')

In [13]:
df_raw

date    HUFL   HULL   MUFL   MULL   LUFL   LULL  \
0      2016-07-01 00:00:00   5.827  2.009  1.599  0.462  4.203  1.340   
1      2016-07-01 01:00:00   5.693  2.076  1.492  0.426  4.142  1.371   
2      2016-07-01 02:00:00   5.157  1.741  1.279  0.355  3.777  1.218   
3      2016-07-01 03:00:00   5.090  1.942  1.279  0.391  3.807  1.279   
4      2016-07-01 04:00:00   5.358  1.942  1.492  0.462  3.868  1.279   
...                    ...     ...    ...    ...    ...    ...    ...   
17415  2018-06-26 15:00:00  -1.674  3.550 -5.615  2.132  3.472  1.523   
17416  2018-06-26 16:00:00  -5.492  4.287 -9.132  2.274  3.533  1.675   
17417  2018-06-26 17:00:00   2.813  3.818 -0.817  2.097  3.716  1.523   
17418  2018-06-26 18:00:00   9.243  3.818  5.472  2.097  3.655  1.432   
17419  2018-06-26 19:00:00  10.114  3.550  6.183  1.564  3.716  1.462   

              OT  
0      29.159000  
1      28.701667  
2      26.872667  
3      24.926334  
4      22.722000  
...          ...  
17415  10.716000  
17416  10.739667  
17417  10.364333  
17418   9.872000  
17419   9.672500  

[17420 rows x 8 columns]

In [14]:
parser = get_args()
args = parser.parse_args([
    '--task_name', 'forecast',
    '--seq_len', '96',
    '--label_len', '0',
    '--pred_len', '12', # pred_len==seq_len if use SegRNN
    '--use_gpu', 'True',
    '--num_workers', '0',
    '--features', 'MS',
    '--inverse',
    '--d_model','512',
    
    '--model', 'AD_MoE', 
    
    '--enc_in', str(df_raw.shape[-1] - 1),
    '--dec_in', str(df_raw.shape[-1] - 1),
    '--c_out', str(df_raw.shape[-1] - 1),   # if use TimesNet
    '--d_ff', '4',
    '--e_layers', '3',
    '--d_layers', '3',
    '--down_sampling_layers', '3',  # if use TimeMixer
    '--down_sampling_window', '2',  # if use TimeMixer
    '--down_sampling_method', 'avg',    # if use TimeMixer
    
    # ModernTCN
    '--ffn_ratio', '1',
    '--patch_size', '16',
    '--patch_stride', '8',
    '--num_blocks', '1',
    '--large_size', '51',
    '--small_size', '5',
    '--dims', '1',
    '--use_multi_scale', 'False',
    '--dropout', '0.05',
    '--head_dropout', '0.0',
    
    '--train_epochs', '300',
    '--batch_size', '64',
    '--learning_rate', '1e-4',
    '--optimizer', 'AdamW',
    '--lradj', 'CosineAnnealingLR',
    '--loss', 'L1',
    
    '--train_ratio', '0.8',
    '--val_ratio', '0.1',
    '--test_ratio', '0.1',
    '--patience', '10',
    '--weight_decay', '0.001',
    '--lr_decay_min_lr', '1e-4',
    '--lr_decay_factor', '0.9',

    
])
# args.dataset = df_raw   # if use Koopa

print_args(args)

Basic Config
  Task Name:          forecast            Is Training:        1                   
  Model ID:           test                Model:              AD_MoE              

Data Loader
  Data:               BFSi                Root Path:          ./dataset/          
  Data Path:          Si.csv              Features:           MS                  
  Target:             OT                  Freq:               h                   
  Checkpoints:        ./checkpoints/      

Forecasting Task
  Seq Len:            96                  Label Len:          0                   
  Pred Len:           12                  Seasonal Patterns:  Monthly             
  Inverse:            1                   

Run Parameters
  Num Workers:        0                   Itr:                1                   
  Train Epochs:       300                 Batch Size:         64                  
  Patience:           10                  Learning Rate:      0.0001              
  Des:                te

In [15]:
setting = get_setting(args)
setting

'forecast_AD_MoE_ds-BFSi_ft-MS_sl-96_ll-0_pl-12__2025-11-17-15-02-37'

In [16]:
exp = Trainer_Forecasting(args, df_raw)
model, history = exp.train(setting)

Use GPU: cuda:0
train 13829
val 1731
test 1731
Training Start:
cuda:0
--------------------------------------------------------------------------------------------
	iters: 100, epoch: 1 | loss: 0.0567914
	speed: 0.0502s/iter; left time: 3248.5739s
	iters: 200, epoch: 1 | loss: 0.0716155
	speed: 0.0503s/iter; left time: 3252.5178s
Epoch: 1 cost time: 10.89237117767334
Epoch: 1, Steps: 216 | Train Loss: 0.0673466 Vali Loss: 0.0359431 Test Loss: 0.0342819
learning rate:  0.0001
Validation loss decreased (inf --> 0.035943).  Saving model ...
--------------------------------------------------------------------------------------------
	iters: 100, epoch: 2 | loss: 0.0499793
	speed: 0.0706s/iter; left time: 4552.0358s
	iters: 200, epoch: 2 | loss: 0.0458598
	speed: 0.0509s/iter; left time: 3277.6201s
Epoch: 2 cost time: 10.959381818771362
Epoch: 2, Steps: 216 | Train Loss: 0.0549582 Vali Loss: 0.0354168 Test Loss: 0.0350950
learning rate:  3.3333333333333333e-06
Validation loss decreased (0.03

In [17]:
preds, trues, full_pred, full_true, metrics, base_dir, svg_path=exp.test(setting)

test 1731
test shape: torch.Size([1731, 12, 1]) torch.Size([1731, 12, 1])
test shape: torch.Size([1731, 12, 1]) torch.Size([1731, 12, 1])
mse:1.6528887748718262, mae:0.8943291902542114, rmse:1.2856472202248275, mape:0.10076088458299637, mspe:0.028922971338033676, dtw:not calculated
Results saved to: test_result/forecast_AD_MoE_ds-BFSi_ft-MS_sl-96_ll-0_pl-12__2025-11-17-15-02-37
±0.05 band hit rate = 0.0996


In [18]:
try:
    # 1. 定义所有容忍度
    tolerances_to_check = [1,3,5]

    # 2. 确保Tensors在CPU上
    if full_true.is_cuda:
        full_true = full_true.cpu()
    if full_pred.is_cuda:
        full_pred = full_pred.cpu()

    # 3. 检查数据点总数
    num_total_predictions = full_true.numel()
    
    if num_total_predictions == 0:
        print("错误：数据为空，无法计算命中率。")
    else:
        # 4. 计算原始尺度上的绝对误差 (一次计算，多次使用)
        abs_error = torch.abs(full_pred - full_true)
        
        print(f"--- 预测任务命中率 (总预测点数: {num_total_predictions}) ---")

        # 5. 循环遍历所有容忍度标准
        for tol in tolerances_to_check:
            # 6. 找到当前容忍度下的命中点
            hits_bool = (abs_error <= tol)

            # 7. 统计命中点数
            num_correct_hits = torch.sum(hits_bool).item()
            
            # 8. 计算最终命中率
            hit_rate_percent = (num_correct_hits / num_total_predictions) * 100.0
            
            # 9. 打印当前容忍度的结果
            print(f"\n  容忍度 ±{tol:.2f}%:")
            print(f"    命中点数: {num_correct_hits}")
            print(f"    最终命中率: {hit_rate_percent:.2f} %")
        
        print("\n-------------------------------------------------")


except Exception as e:
    print(f"计算过程中发生错误: {e}")
    print("请确保 'full_true' 和 'full_pred' 是PyTorch Tensors。")

--- 预测任务命中率 (总预测点数: 1838) ---

  容忍度 ±1.00%:
    命中点数: 1356
    最终命中率: 73.78 %

  容忍度 ±3.00%:
    命中点数: 1803
    最终命中率: 98.10 %

  容忍度 ±5.00%:
    命中点数: 1835
    最终命中率: 99.84 %

-------------------------------------------------
